In [ ]:
!pip install --upgrade transformers scipy ftfy accelerate diffusers

{'ori_sentence': 'an apple', 'tar_sentence': 'an apple and a lake', 'oo': 'apple', 'to': 'lake'}


In [ ]:
import os 
import json
import torch

In [ ]:
os.mkdir("adv_prompts/sentence")

# Load sentence 
with open('dataset/sentence1.json', 'r') as file:
    data = json.load(file)
# Copy 
with open('adv_prompts/sentence/sentence1.json', 'w') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

oo = data['oo']
to = data['to']
ori_sentence = data['ori_sentence'] 
tar_sentence = data['tar_sentence'] 
print('Original object:', oo)
print('Target object:', to)
print('Original sentence:', ori_sentence)
print('Target sentence:', tar_sentence)

In [ ]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer

len_prompt = 5

tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = CLIPTextModel.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = text_encoder.to('cuda')